In [1]:
import psycopg2

In [ ]:
# Connecter à PostgreSQL
def create_connection():
    try:
        conn = psycopg2.connect(
            dbname="documents", 
            user="avnadmin", 
            password="AVNS_l7Jb2H8TcLKMke_Jm_j", 
            host="pg-328cb18b-hamdounechaymae-7b98.h.aivencloud.com", 
            port="10809"
        )
        return conn
    except Exception as e:
        print(f"Erreur de connexion à la base de données: {e}")
        return None

In [ ]:
def create_tables():
    commands = (
        """
        CREATE TABLE doc_list (
            id SERIAL PRIMARY KEY,
            title VARCHAR(255) NOT NULL
        )
        """,
        """
        CREATE TABLE term_dict (
            termID SERIAL PRIMARY KEY,
            term VARCHAR(255) NOT NULL
        )
        """,
        """
        CREATE TABLE inverted_index (
            termID INT,
            docID INT,
            freq INT,
            tf_idf FLOAT,
            positions TEXT,
            FOREIGN KEY (termID) REFERENCES term_dict(termID),
            FOREIGN KEY (docID) REFERENCES doc_list(id)
        )
        """
    )
    conn = create_connection()
    if conn is not None:
        try:
            cur = conn.cursor()
            for command in commands:
                cur.execute(command)
            cur.close()
            conn.commit()
        except Exception as e:
            print(f"Erreur lors de la création des tables: {e}")
        finally:
            conn.close()
    else:
        print("Erreur de connexion à la base de données.")
        
create_tables()

In [ ]:
def check_tables():
    conn = create_connection()
    if conn is not None:
        try:
            cur = conn.cursor()
            cur.execute("""
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'public'
            """)
            tables = cur.fetchall()
            print("Tables in the database:")
            for table in tables:
                print(table[0])
            cur.close()
        except Exception as e:
            print(f"Erreur lors de la vérification des tables: {e}")
        finally:
            conn.close()
    else:
        print("Erreur de connexion à la base de données.")

# Appeler la fonction pour vérifier les tables
check_tables()

In [ ]:
def insert_data_from_csv(table_name, csv_file_path):
    conn = create_connection()
    if conn is not None:
        try:
            cur = conn.cursor()
            with open(csv_file_path, 'r') as f:
                next(f)  # Skip the header row
                cur.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV HEADER", f)
            conn.commit()
            cur.close()
            print(f"Data inserted into {table_name} from {csv_file_path}")
        except Exception as e:
            print(f"Erreur lors de l'insertion des données: {e}")
        finally:
            conn.close()
    else:
        print("Erreur de connexion à la base de données.")

In [ ]:
# def insert_into_doc_list(id, title):
#     conn = create_connection()
#     if conn is not None:
#         try:
#             cur = conn.cursor()
#             cur.execute("INSERT INTO doc_list (id, title) VALUES (%s, %s)", (id, title))
#             conn.commit()
#             cur.close()
#             print(f"Inserted into doc_list: ({id}, {title})")
#         except Exception as e:
#             print(f"Erreur lors de l'insertion dans doc_list: {e}")
#         finally:
#             conn.close()
#     else:
#         print("Erreur de connexion à la base de données.")

# def insert_into_term_dict(termID, term):
#     conn = create_connection()
#     if conn is not None:
#         try:
#             cur = conn.cursor()
#             cur.execute("INSERT INTO term_dict (termID, term) VALUES (%s, %s)", (termID, term))
#             conn.commit()
#             cur.close()
#             print(f"Inserted into term_dict: ({termID}, {term})")
#         except Exception as e:
#             print(f"Erreur lors de l'insertion dans term_dict: {e}")
#         finally:
#             conn.close()
#     else:
#         print("Erreur de connexion à la base de données.")

# # Insert data into doc_list
# insert_into_doc_list(0, "Predisposing factors for incomplete spontaneous recovery after Parsonage-Turner Syndrome.")

# # Insert data into term_dict
# insert_into_term_dict(0, "2000")

In [ ]:
insert_data_from_csv('doc_list', './doc_list.csv')
insert_data_from_csv('term_dict', './term_dict.csv')
insert_data_from_csv('inverted_index', './inverted_index.csv')

In [ ]:
def check_table_contents(table_name):
    conn = create_connection()
    if conn is not None:
        try:
            cur = conn.cursor()
            cur.execute(f"SELECT * FROM {table_name}")
            rows = cur.fetchall()
            print(f"Contents of {table_name}:")
            for row in rows:
                print(row)
            cur.close()
        except Exception as e:
            print(f"Erreur lors de la vérification des contenus de la table {table_name}: {e}")
        finally:
            conn.close()
    else:
        print("Erreur de connexion à la base de données.")

# Example usage
check_table_contents('doc_list')
check_table_contents('term_dict')
check_table_contents('inverted_index')